### Importing the Necessary Moduels

In [5]:
import pandas as pd

# Load the Excel file to inspect its contents
file_path = 'supply_chain_data.xlsx'
excel_data = pd.ExcelFile(file_path)

# Display the sheet names to understand the structure of the data
excel_data.sheet_names


['OrderList',
 'FreightRates',
 'WhCosts',
 'WhCapacities',
 'ProductsPerPlant',
 'VmiCustomers',
 'PlantPorts']

In [6]:
# Load data from all sheets into a dictionary of DataFrames
data = {sheet: excel_data.parse(sheet) for sheet in excel_data.sheet_names}

# Display the first few rows of each sheet to understand their structure
{sheet: df.head() for sheet, df in data.items()}


{'OrderList':        Order ID Order Date Origin Port Carrier  TPT Service Level  \
 0  1.447296e+09 2013-05-26      PORT09   V44_3    1           CRF   
 1  1.447158e+09 2013-05-26      PORT09   V44_3    1           CRF   
 2  1.447139e+09 2013-05-26      PORT09   V44_3    1           CRF   
 3  1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 4  1.447364e+09 2013-05-26      PORT09   V44_3    1           CRF   
 
    Ship ahead day count  Ship Late Day count   Customer  Product ID  \
 0                     3                    0  V55555_53     1700106   
 1                     3                    0  V55555_53     1700106   
 2                     3                    0  V55555_53     1700106   
 3                     3                    0  V55555_53     1700106   
 4                     3                    0  V55555_53     1700106   
 
   Plant Code Destination Port  Unit quantity  Weight  
 0    PLANT16           PORT09            808   14.30  
 1    PLANT16      

#### Removing whitespaces from the columns of the Freight Rates table

In [14]:
data['FreightRates']['orig_port_cd'] = data['FreightRates']['orig_port_cd'].str.strip()
data['FreightRates']['dest_port_cd'] = data['FreightRates']['dest_port_cd'].str.strip()
data['FreightRates']['Carrier'] = data['FreightRates']['Carrier'].str.strip()
data['FreightRates']['svc_cd'] = data['FreightRates']['svc_cd'].str.strip()
data['FreightRates']['mode_dsc'] = data['FreightRates']['mode_dsc'].str.strip()


In [15]:
from pulp import *

#### Decision Variables x_ki and y_kcpjstm

In [39]:
# Generate the x_ki variable
x_ki = pulp.LpVariable.dicts(
    "x_ki",
    (
    (k, i)
    for k in data['OrderList'].index[:1] # For every order number in the OrderList table
    for i in data['PlantPorts']['Plant Code'].unique() # For every unique Warehouse/Plant in the PlantPorts table
    ),
    cat = "Binary"
)

In [40]:
x_ki

{(0, 'PLANT01'): x_ki_(0,_'PLANT01'),
 (0, 'PLANT02'): x_ki_(0,_'PLANT02'),
 (0, 'PLANT03'): x_ki_(0,_'PLANT03'),
 (0, 'PLANT04'): x_ki_(0,_'PLANT04'),
 (0, 'PLANT05'): x_ki_(0,_'PLANT05'),
 (0, 'PLANT06'): x_ki_(0,_'PLANT06'),
 (0, 'PLANT07'): x_ki_(0,_'PLANT07'),
 (0, 'PLANT08'): x_ki_(0,_'PLANT08'),
 (0, 'PLANT09'): x_ki_(0,_'PLANT09'),
 (0, 'PLANT10'): x_ki_(0,_'PLANT10'),
 (0, 'PLANT11'): x_ki_(0,_'PLANT11'),
 (0, 'PLANT12'): x_ki_(0,_'PLANT12'),
 (0, 'PLANT13'): x_ki_(0,_'PLANT13'),
 (0, 'PLANT14'): x_ki_(0,_'PLANT14'),
 (0, 'PLANT15'): x_ki_(0,_'PLANT15'),
 (0, 'PLANT16'): x_ki_(0,_'PLANT16'),
 (0, 'PLANT17'): x_ki_(0,_'PLANT17'),
 (0, 'PLANT18'): x_ki_(0,_'PLANT18'),
 (0, 'PLANT19'): x_ki_(0,_'PLANT19')}

In [16]:
# Generate the y_kcpjstm variable
y_kcpjstm = pulp.LpVariable.dicts(
    "y_kcpjstm",
    (
        (k, c, p, j, s, t, m)  # Exclude 'k' from FreightRates indexing
        for k in data['OrderList'].index[:1]  # Orders
        for c, group in data['FreightRates'].groupby('Carrier')  # Iterate over each Carrier
        for p, j, s, t, m in group[['orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']].drop_duplicates().itertuples(index=False)
    ),
    cat="Binary",
)


In [17]:
len(y_kcpjstm)

76

In [18]:
y_kcpjstm

{(0,
  'V444_0',
  'PORT02',
  'PORT09',
  'DTD',
  4,
  'AIR'): y_kcpjstm_(0,_'V444_0',_'PORT02',_'PORT09',_'DTD',_4,_'AIR'),
 (0,
  'V444_0',
  'PORT04',
  'PORT09',
  'DTD',
  3,
  'AIR'): y_kcpjstm_(0,_'V444_0',_'PORT04',_'PORT09',_'DTD',_3,_'AIR'),
 (0,
  'V444_0',
  'PORT04',
  'PORT09',
  'DTP',
  3,
  'AIR'): y_kcpjstm_(0,_'V444_0',_'PORT04',_'PORT09',_'DTP',_3,_'AIR'),
 (0,
  'V444_0',
  'PORT04',
  'PORT09',
  'DTD',
  2,
  'AIR'): y_kcpjstm_(0,_'V444_0',_'PORT04',_'PORT09',_'DTD',_2,_'AIR'),
 (0,
  'V444_0',
  'PORT04',
  'PORT09',
  'DTP',
  2,
  'AIR'): y_kcpjstm_(0,_'V444_0',_'PORT04',_'PORT09',_'DTP',_2,_'AIR'),
 (0,
  'V444_0',
  'PORT03',
  'PORT09',
  'DTD',
  1,
  'GROUND'): y_kcpjstm_(0,_'V444_0',_'PORT03',_'PORT09',_'DTD',_1,_'GROUND'),
 (0,
  'V444_0',
  'PORT09',
  'PORT09',
  'DTP',
  0,
  'GROUND'): y_kcpjstm_(0,_'V444_0',_'PORT09',_'PORT09',_'DTP',_0,_'GROUND'),
 (0,
  'V444_0',
  'PORT09',
  'PORT09',
  'DTD',
  0,
  'GROUND'): y_kcpjstm_(0,_'V444_0',_'PORT09

In [19]:
(k, c, p, j, s, t, m) = (0, 'V444_1', 'PORT10', 'PORT09', 'DTD', 5, 'AIR')

In [20]:
y_kcpjstm[k, c, p, j, s, t, m]

y_kcpjstm_(0,_'V444_1',_'PORT10',_'PORT09',_'DTD',_5,_'AIR')

In [31]:
data['OrderList'].head()

,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.30
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.94
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.20
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.16
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.34


In [23]:
# The first row in the OrderList table has weight = 14.3 which does not match any rows in the Freight Rates table
data['FreightRates'].set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']).loc[(c, p, j, s, t, m)].query("minm_wgh_qty <= @data['OrderList'].loc[@k]['Weight'] <= max_wgh_qty")['rate']#.iloc[0] * data['OrderList'].loc[k]['Weight']

/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/1952414235.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  data['FreightRates'].set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']).loc[(c, p, j, s, t, m)].query("minm_wgh_qty <= @data['OrderList'].loc[@k]['Weight'] <= max_wgh_qty")['rate']#.iloc[0] * data['OrderList'].loc[k]['Weight']


Series([], Name: rate, dtype: float64)

#### Transportation Cost Variable

In [35]:
# Define transportation cost
transportation_cost = pulp.lpSum(
    y_kcpjstm[k, c, p, j, s, t, m]
    * max(
        # Minimum cost
        data['FreightRates']
        .set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc'])
        .loc[(c, p, j, s, t, m)]['minimum cost'].iloc[0],

        # Rate multiplied by weight
        (
            data['FreightRates']
            .set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc'])
            .loc[(c, p, j, s, t, m)]
            .query("minm_wgh_qty <= @data['OrderList'].loc[@k]['Weight'] <= max_wgh_qty")['rate'].iloc[0]
            * data['OrderList'].loc[k]['Weight']
        )
        if not data['FreightRates']
        .set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc'])
        .loc[(c, p, j, s, t, m)]
        .query("minm_wgh_qty <= @data['OrderList'].loc[@k]['Weight'] <= max_wgh_qty").empty
        else data['FreightRates']
        .set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc'])
        .loc[(c, p, j, s, t, m)]['rate'].mean()
    )
    for k in data['OrderList'].index[:1]
    for c, group in data['FreightRates'].groupby('Carrier')
    for p, j, s, t, m in group[['orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']]
    .drop_duplicates().itertuples(index=False)
)


/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/2533690252.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/2533690252.py:18: PerformanceWarning: indexing past lexsort depth may impact performance.
  if not data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/2533690252.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/2533690252.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/2533690252.py:18: PerformanceWarning: indexing past lexsort depth may impact performance.
  if not data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/2533690252.py:12: PerformanceWarn

In [36]:
len(transportation_cost)

76

In [37]:
for k, v in transportation_cost.items():
    print(k, v)

y_kcpjstm_(0,_'V444_0',_'PORT02',_'PORT09',_'DTD',_4,_'AIR') 23.8384
y_kcpjstm_(0,_'V444_0',_'PORT04',_'PORT09',_'DTD',_3,_'AIR') 3.4552
y_kcpjstm_(0,_'V444_0',_'PORT04',_'PORT09',_'DTP',_3,_'AIR') 1.4992
y_kcpjstm_(0,_'V444_0',_'PORT04',_'PORT09',_'DTD',_2,_'AIR') 3.4552
y_kcpjstm_(0,_'V444_0',_'PORT04',_'PORT09',_'DTP',_2,_'AIR') 1.4992
y_kcpjstm_(0,_'V444_0',_'PORT03',_'PORT09',_'DTD',_1,_'GROUND') 231.69432000000003
y_kcpjstm_(0,_'V444_0',_'PORT09',_'PORT09',_'DTP',_0,_'GROUND') 175.58112
y_kcpjstm_(0,_'V444_0',_'PORT09',_'PORT09',_'DTD',_0,_'GROUND') 189.88112
y_kcpjstm_(0,_'V444_1',_'PORT10',_'PORT09',_'DTD',_5,_'AIR') 1.5884
y_kcpjstm_(0,_'V444_1',_'PORT10',_'PORT09',_'DTD',_3,_'AIR') 1.5884
y_kcpjstm_(0,_'V444_1',_'PORT06',_'PORT09',_'DTD',_2,_'AIR') 20.346040000000002
y_kcpjstm_(0,_'V444_1',_'PORT04',_'PORT09',_'DTD',_2,_'AIR') 1.202
y_kcpjstm_(0,_'V444_1',_'PORT05',_'PORT09',_'DTD',_2,_'AIR') 1.73996
y_kcpjstm_(0,_'V444_1',_'PORT04',_'PORT09',_'DTD',_1,_'AIR') 1.202
y_kcpjstm

#### Warehouse Cost Variable

In [41]:
warehouse_cost = pulp.lpSum(
    x_ki[k, i] * data['OrderList'].loc[k, 'Unit quantity'] * data['WhCosts'].set_index('WH').loc[i, 'Cost/unit']
    for k in data['OrderList'].index[:1]
    for i in data['WhCosts']['WH']
)

In [47]:
print(len(warehouse_cost))
for k,v in warehouse_cost.items():
    print(k,v)


19
x_ki_(0,_'PLANT15') 1143.3712083636265
x_ki_(0,_'PLANT17') 346.5888423680831
x_ki_(0,_'PLANT18') 1645.293037921104
x_ki_(0,_'PLANT05') 394.4206626981252
x_ki_(0,_'PLANT02') 385.8228432931957
x_ki_(0,_'PLANT01') 458.1167574230859
x_ki_(0,_'PLANT06') 447.7031828816371
x_ki_(0,_'PLANT10') 398.81417397421984
x_ki_(0,_'PLANT07') 300.11054143976037
x_ki_(0,_'PLANT14') 512.5384280653706
x_ki_(0,_'PLANT16') 1551.2044663447662
x_ki_(0,_'PLANT12') 624.6904898088517
x_ki_(0,_'PLANT11') 448.63981068457304
x_ki_(0,_'PLANT09') 375.7774466854115
x_ki_(0,_'PLANT03') 418.14152843337314
x_ki_(0,_'PLANT13') 379.5233017178626
x_ki_(0,_'PLANT19') 516.9281160125348
x_ki_(0,_'PLANT08') 422.46869775054705
x_ki_(0,_'PLANT04') 346.2306624797596


In [57]:
data['FreightRates'].set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']).loc[(c, p, j, s, t, m)]['max_wgh_qty'].max()


/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/1775648345.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  data['FreightRates'].set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']).loc[(c, p, j, s, t, m)]['max_wgh_qty'].max()


np.float64(99999.99)

In [69]:
# 7. Restrict warehouses to ship orders only through allowed ports
for k in data['OrderList'].index[:1]:
    for i in data['WhCosts']['WH']:
        # Get the allowed ports for the current warehouse
            allowed_ports = data['PlantPorts'].set_index('Plant Code').loc[i, 'Port']
            print(allowed_ports)
            count = 0
            allowed = 0
            not_allowed = 0
            for c, group in data['FreightRates'].groupby('Carrier'):
                for (p, j, s, t, m) in group[['orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']].drop_duplicates().itertuples(index=False):
                    count += 1
                    if p in allowed_ports:
                        allowed += 1
                        print(f"{y_kcpjstm[k, c, p, j, s, t, m]} compatible with {x_ki[k, i]}")
                    else:
                         not_allowed +=1
print(count, allowed, not_allowed)

PORT08
y_kcpjstm_(0,_'V444_2',_'PORT08',_'PORT09',_'DTD',_6,_'AIR') compatible with x_ki_(0,_'PLANT15')
y_kcpjstm_(0,_'V444_2',_'PORT08',_'PORT09',_'DTD',_5,_'AIR') compatible with x_ki_(0,_'PLANT15')
y_kcpjstm_(0,_'V444_2',_'PORT08',_'PORT09',_'DTP',_5,_'AIR') compatible with x_ki_(0,_'PLANT15')
y_kcpjstm_(0,_'V444_6',_'PORT08',_'PORT09',_'DTD',_2,_'AIR') compatible with x_ki_(0,_'PLANT15')
y_kcpjstm_(0,_'V444_9',_'PORT08',_'PORT09',_'DTD',_14,_'AIR') compatible with x_ki_(0,_'PLANT15')
PORT10
y_kcpjstm_(0,_'V444_1',_'PORT10',_'PORT09',_'DTD',_5,_'AIR') compatible with x_ki_(0,_'PLANT17')
y_kcpjstm_(0,_'V444_1',_'PORT10',_'PORT09',_'DTD',_3,_'AIR') compatible with x_ki_(0,_'PLANT17')
y_kcpjstm_(0,_'V444_2',_'PORT10',_'PORT09',_'DTD',_6,_'AIR') compatible with x_ki_(0,_'PLANT17')
y_kcpjstm_(0,_'V444_2',_'PORT10',_'PORT09',_'DTD',_5,_'AIR') compatible with x_ki_(0,_'PLANT17')
y_kcpjstm_(0,_'V444_2',_'PORT10',_'PORT09',_'DTD',_3,_'AIR') compatible with x_ki_(0,_'PLANT17')
y_kcpjstm_(0,_'

In [5]:
import pulp

# Initialize the optimization model
model = pulp.LpProblem("SupplyChainOptimization", pulp.LpMinimize)

# --- Decision Variables ---
# 1. x_ki: Binary variable for order k assigned to warehouse i
# 2. y_kpjcstm: Binary variable for order k assigned to shipping lane p-j-c-s-t-m
# x_ki = pulp.LpVariable.dicts("x_ki", ((k, i) for k in data['OrderList'].index for i in data['WhCosts']['WH']), cat="Binary")
# y_kpjcstm = pulp.LpVariable.dicts(
#     "y_kpjcstm",
#     (
#         (k, p, j, c, s, t, m)
#         for k in data['OrderList'].index
#         for p in data['PlantPorts']['Port'].unique()
#         for j in data['OrderList']['Destination Port'].unique()
#         for c in data['FreightRates']['Carrier'].unique()
#         for s in data['FreightRates']['svc_cd'].unique()
#         for t in data['FreightRates']['tpt_day_cnt'].unique()
#         for m in data['FreightRates']['mode_dsc'].unique()
#     ),
#     cat="Binary",
# )

# --- Objective Function ---
# Minimize total cost = Warehouse cost + Transportation cost
# warehouse_cost = pulp.lpSum(
#     x_ki[k, i] * data['OrderList'].loc[k, 'Unit quantity'] * data['WhCosts'].set_index('WH').loc[i, 'Cost/unit']
#     for k in data['OrderList'].index
#     for i in data['WhCosts']['WH']
# )

# transportation_cost = pulp.lpSum(
#     y_kpjcstm[k, p, j, c, s, t, m]
#     * max(
#         data['FreightRates'].set_index(['orig_port_cd', 'dest_port_cd', 'Carrier']).loc[(p, j, c), 'minimum cost'],
#         data['FreightRates'].set_index(['orig_port_cd', 'dest_port_cd', 'Carrier']).loc[(p, j, c), 'rate']
#         * data['OrderList'].loc[k, 'Weight'],
#     )
#     for k in data['OrderList'].index
#     for p in data['PlantPorts']['Port']
#     for j in data['OrderList']['Destination Port'].unique()
#     for c in data['FreightRates']['Carrier'].unique()
#     for s in data['FreightRates']['svc_cd'].unique()
#     for t in data['FreightRates']['tpt_day_cnt'].unique()
#     for m in data['FreightRates']['mode_dsc'].unique()
# )

model += warehouse_cost + transportation_cost

# --- Constraints ---
# 1. Each order is assigned to exactly one warehouse
for k in data['OrderList'].index:
    model += pulp.lpSum(x_ki[k, i] for i in data['WhCosts']['WH']) == 1

# 2. Each order is assigned to one shipping lane
for k in data['OrderList'].index:
    model += pulp.lpSum(
        y_kcpjstm[k, c, p, j, s, t, m]
        for c, group in data['FreightRates'].groupby('Carrier')  # Iterate over each Carrier
        for p, j, s, t, m in group[['orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']].drop_duplicates().itertuples(index=False)
    ) == 1

# 3. Warehouse capacity constraints
for i in data['WhCapacities']['Plant ID']:
    model += pulp.lpSum(x_ki[k, i] for k in data['OrderList'].index) <= data['WhCapacities'].set_index('Plant ID').loc[i, 'Daily Capacity ']

# 4. Shipping lane weight constraints
for (p, j, c, s, t, m) in data['FreightRates'][['orig_port_cd', 'dest_port_cd', 'Carrier', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']].itertuples(index=False):
    model += pulp.lpSum(
        y_kpjcstm[k, p, j, c, s, t, m] * data['OrderList'].loc[k, 'Weight'] for k in data['OrderList'].index
    ) <= data['FreightRates'].set_index(['orig_port_cd', 'dest_port_cd', 'Carrier']).loc[(p, j, c), 'max_wgh_qty']

# 5. Product compatibility with warehouses
for k in data['OrderList'].index:
    for i in data['WhCosts']['WH']:
        if data['OrderList'].loc[k, 'Product ID'] not in data['ProductsPerPlant'].set_index('Plant Code').loc[i, 'Product ID']:
            model += x_ki[k, i] == 0

# 6. Customer-warehouse restrictions
for k in data['OrderList'].index:
    for i in data['WhCosts']['WH']:
        # Check if the warehouse-customer combination is in VmiCustomers
        if i in data['VmiCustomers']['Plant Code'].unique():
            allowed_customers = data['VmiCustomers'].set_index('Plant Code').loc[i, 'Customers']
            if data['OrderList'].loc[k, 'Customer'] in allowed_customers:
                model += x_ki[k, i] == 1

# 7. Restrict warehouses to ship orders only through allowed ports
for k in data['OrderList'].index:
    for i in data['WhCosts']['WH']:
        # Get the allowed ports for the current warehouse
        if i in data['PlantPorts']['Plant Code'].unique():
            allowed_ports = data['PlantPorts'].set_index('Plant Code').loc[i, 'Port']
            for p in data['PlantPorts']['Port'].unique():
                if p not in allowed_ports:
                    for j in data['OrderList']['Destination Port'].unique():
                        for c in data['FreightRates']['Carrier'].unique():
                            for s in data['FreightRates']['svc_cd'].unique():
                                for t in data['FreightRates']['tpt_day_cnt'].unique():
                                    for m in data['FreightRates']['mode_dsc'].unique():
                                        # Enforce the constraint
                                        model += y_kpjcstm[k, p, j, c, s, t, m] * x_ki[k, i] == 0


# Specify the path to Gurobi
path_to_Gurobi = '/Library/gurobi1003/macos_universal2/bin/gurobi_cl'

# Solve the problem using Gurobi
model.solve(GUROBI_CMD(path=path_to_Gurobi))


# # Output the results
# status = pulp.LpStatus[model.status]
# objective_value = pulp.value(model.objective)
# (status, objective_value)


In [93]:
import pulp

# Initialize the optimization model
model = pulp.LpProblem("SupplyChainOptimization", pulp.LpMinimize)

# --- Decision Variables ---
# 1. x_ki: Binary variable for order k assigned to warehouse i
x_ki = pulp.LpVariable.dicts(
    "x_ki",
    (
    (k, i)
    for k in data['OrderList'].index[:1] # For every order number in the OrderList table
    for i in data['PlantPorts']['Plant Code'].unique() # For every unique Warehouse/Plant in the PlantPorts table
    ),
    cat = "Binary"
)

# 2. y_kcpjstm: Binary variable for order k assigned to shipping lane c-p-j-s-t-m
y_kcpjstm = pulp.LpVariable.dicts(
    "y_kcpjstm",
    (
        (k, c, p, j, s, t, m)  # Exclude 'k' from FreightRates indexing
        for k in data['OrderList'].index[:1]  # Orders
        for c, group in data['FreightRates'].groupby('Carrier')  # Iterate over each Carrier
        for p, j, s, t, m in group[['orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']].drop_duplicates().itertuples(index=False)
    ),
    cat="Binary",
)

# --- Objective Function ---

# Minimize total cost = Warehouse cost + Transportation cost
warehouse_cost = pulp.lpSum(
    x_ki[k, i] * data['OrderList'].loc[k, 'Unit quantity'] * data['WhCosts'].set_index('WH').loc[i, 'Cost/unit']
    for k in data['OrderList'].index[:1]
    for i in data['WhCosts']['WH']
)

transportation_cost = pulp.lpSum(
    y_kcpjstm[k, c, p, j, s, t, m]
    * max(
        # Minimum cost
        data['FreightRates']
        .set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc'])
        .loc[(c, p, j, s, t, m)]['minimum cost'].iloc[0],

        # Rate multiplied by weight
        (
            data['FreightRates']
            .set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc'])
            .loc[(c, p, j, s, t, m)]
            .query("minm_wgh_qty <= @data['OrderList'].loc[@k]['Weight'] <= max_wgh_qty")['rate'].iloc[0]
            * data['OrderList'].loc[k]['Weight']
        )
        if not data['FreightRates']
        .set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc'])
        .loc[(c, p, j, s, t, m)]
        .query("minm_wgh_qty <= @data['OrderList'].loc[@k]['Weight'] <= max_wgh_qty").empty
        else data['FreightRates']
        .set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc'])
        .loc[(c, p, j, s, t, m)]['rate'].mean()
    )
    for k in data['OrderList'].index[:1]
    for c, group in data['FreightRates'].groupby('Carrier')
    for p, j, s, t, m in group[['orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']]
    .drop_duplicates().itertuples(index=False)
)

model += warehouse_cost + transportation_cost

# --- Constraints ---
# 1. Each order is assigned to exactly one warehouse
for k in data['OrderList'].index[:1]:
    model += pulp.lpSum(x_ki[k, i] for i in data['WhCosts']['WH']) == 1

# 2. Each order is assigned to one shipping lane
for k in data['OrderList'].index[:1]:
    model += pulp.lpSum(
        y_kcpjstm[k, c, p, j, s, t, m]
        for c, group in data['FreightRates'].groupby('Carrier')  # Iterate over each Carrier
        for p, j, s, t, m in group[['orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']]
        .drop_duplicates().itertuples(index=False)
    ) == 1

# 3. Warehouse capacity constraints
for i in data['WhCapacities']['Plant ID']:
    model += pulp.lpSum(x_ki[k, i] for k in data['OrderList'].index[:1]) <= data['WhCapacities'].set_index('Plant ID').loc[i, 'Daily Capacity ']

# 4. Shipping lane weight constraints
for c, group in data['FreightRates'].groupby('Carrier'):
    for (p, j, s, t, m) in group[['orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']].drop_duplicates().itertuples(index=False):
        model += pulp.lpSum(
            y_kcpjstm[k, c, p, j, s, t, m] * data['OrderList'].loc[k, 'Weight'] for k in data['OrderList'].index[:1]
            ) <= data['FreightRates'].set_index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']).loc[(c, p, j, s, t, m), 'max_wgh_qty'].max()

# 5. Product compatibility with warehouses
for k in data['OrderList'].index[:1]:
    for i in data['WhCosts']['WH']:
        products = data['ProductsPerPlant'].set_index('Plant Code').loc[i, 'Product ID']
        products = [products] if not isinstance(products, pd.Series) else products.to_list()
        if data['OrderList'].loc[k, 'Product ID'] not in products:
            model += x_ki[k, i] == 0

# 6. Customer-warehouse restrictions
for k in data['OrderList'].index[:1]:
    for i in data['WhCosts']['WH']:
        # Check if the warehouse-customer combination is in VmiCustomers
        if i in data['VmiCustomers']['Plant Code'].unique():
            allowed_customers = data['VmiCustomers'].set_index('Plant Code').loc[i, 'Customers']
            if data['OrderList'].loc[k, 'Customer'] not in allowed_customers:
                model += x_ki[k, i] == 0

# 7. Restrict warehouses to ship orders only through allowed ports
for k in data['OrderList'].index[:1]:
    for i in data['WhCosts']['WH']:
        # Get the allowed ports for the current warehouse
            allowed_ports = data['PlantPorts'].set_index('Plant Code').loc[i, 'Port']
            for c, group in data['FreightRates'].groubpy('Carrier'):
                for (p, j, s, t, m) in group[['orig_port_cd', 'dest_port_cd', 'svc_cd', 'tpt_day_cnt', 'mode_dsc']].drop_duplicates().itertuples(index=False):
                    if p not in allowed_ports:
                        # Enforce the constraint
                        model += y_kcpjstm[k, c, p, j, s, t, m] * x_ki[k, i] == 0



# Specify the path to Gurobi
path_to_Gurobi = '/Library/gurobi1003/macos_universal2/bin/gurobi_cl'

# Solve the problem using Gurobi
model.solve(GUROBI_CMD(path=path_to_Gurobi))


# # Output the results
# status = pulp.LpStatus[model.status]
# objective_value = pulp.value(model.objective)
# (status, objective_value)


/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/3084980721.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
  data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/3084980721.py:55: PerformanceWarning: indexing past lexsort depth may impact performance.
  if not data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/3084980721.py:49: PerformanceWarning: indexing past lexsort depth may impact performance.
  data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/3084980721.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
  data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/3084980721.py:55: PerformanceWarning: indexing past lexsort depth may impact performance.
  if not data['FreightRates']
/var/folders/s9/cygvrxqn4xd69ndg0hg5k56m0000gn/T/ipykernel_13158/3084980721.py:49: PerformanceWa

KeyError: 'PLANT19'

In [92]:
products = data['ProductsPerPlant'].set_index('Plant Code').loc[i, 'Product ID']
products = [products] if not isinstance(products, pd.Series) else products.to_list()

if data['OrderList'].loc[k, 'Product ID'] not in products:
    print(True)
else:
    print(False)


True


In [79]:
data['ProductsPerPlant'].set_index('Plant Code').loc[i, 'Product ID'].tolist()

1698815

In [76]:
i

'PLANT15'